In [100]:
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import numpy as np
import re
import urllib.request as req
warnings.filterwarnings('ignore')

In [2]:
# import full datasets
# property_2019_full    = pd.read_csv('data/property-assessment-fy2019.csv')
property_2018_full    = pd.read_csv('data/property-assessment-fy2018.csv')
# property_2017_full    = pd.read_csv('data/property-assessment-fy2017.csv')
# property_2016_full    = pd.read_csv('data/property-assessment-fy2016.csv')
# property_2015_full    = pd.read_csv('data/property-assessment-fy2015.csv')

property_full = property_2018_full

In [3]:
# list of columns to save for properties
property_cols = ['ST_NUM','ST_NAME','ST_NAME_SUF','ZIPCODE','AV_TOTAL']

# drop all columns not in list (keep _ at end of name to show not fully clean yet)
property_ = property_full[property_full.columns[property_full.columns.isin(property_cols)]]

# drop rows with no street number because then the lat long is very off
property_df = property_[property_['ST_NUM'].str.strip().astype(bool)]

In [4]:
# get adresses without prices
addresses_df = property_df[property_cols[:-1]]

In [116]:
for i,row in addresses_df.iterrows():
        
    # if zipcode doesnt exist still keep row bc lat long findable
    if np.isnan(row.ZIPCODE) == True: row.ZIPCODE = ' '
    else: row.ZIPCODE = str(row.ZIPCODE)[:-2]
        
    # convert everything to strings
    stnum   = str(row.ST_NUM)
    stname  = str(row.ST_NAME)
    stsuf   = str(row.ST_NAME_SUF)
    zipcode = str(row.ZIPCODE)
        
    # combine columns into one address
    address = stnum.split(' ')[0] + '+' + stname + '+' + stsuf + '+' + zipcode + '+Boston,+MA,+USA'
    addresses_df.at[i,'ADDRESS'] = address

In [18]:
# addresses_df.to_csv('data/addresses_df.csv')

In [25]:
addresses_df.ADDRESS.to_csv('data/addresses_df.csv',header='ADDRESSES')

In [113]:
test_df = addresses_df.loc[0:2750]

In [39]:
test_df.ADDRESS.to_csv('data/test_df.csv',header='ADDRESSES')

In [115]:
test_df.ADDRESS.head()

0       104+PUTNAM+ST+2128 Boston,+MA,+USA
1    197+LEXINGTON+ST+2128 Boston,+MA,+USA
2    199+LEXINGTON+ST+2128 Boston,+MA,+USA
3    201+LEXINGTON+ST+2128 Boston,+MA,+USA
4    203+LEXINGTON+ST+2128 Boston,+MA,+USA
Name: ADDRESS, dtype: object

In [107]:
%time
querie = '199+LEXINGTON+ST+2128+Boston+MA+USA'
# querie = '22+hillside+terrace,+belmont,+MA,+024780,+USA'
contents = req.urlopen('https://nominatim.openstreetmap.org/search?q='+querie+'&format=geocodejson').read()
longlat = re.findall(r'[\w\.-]+,[\w\.-]+', str(contents))

if bool(longlat): 
    long = longlat[0].split(',')[0]
    lat  = longlat[0].split(',')[1]
    
long, lat

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.91 µs


('-71.0324512142857', '42.3795167142857')

In [110]:
'abc'.split(' ')[0]

'abc'